In [4]:
import numpy as np
import pandas as pd
import datetime
from multiprocessing import Pool as ProcessPool

import warnings 
warnings.filterwarnings('ignore')


In [5]:
df_user_testA = pd.read_csv('..\\data\\ECommAI_EUIR_round1_testA_20190701\\user.csv',header=None)
df_user_testA.columns = ['userID','gender','age','purchaseLevel']
df_item_testA = pd.read_csv('..\\data\\ECommAI_EUIR_round1_testA_20190701\\item.csv',header=None)
df_item_testA.columns = ['itemID','categoryID','shopID','brandID']
df_log_testA = pd.read_csv('..\\data\\ECommAI_EUIR_round1_testA_20190701\\user_behavior.csv',header=None)
df_log_testA.columns = ['userID','itemID','behavior','timestap']

df_user_testB = pd.read_csv('..\\data\\ECommAI_EUIR_round1_testB_20190809\\user.csv',header=None)
df_user_testB.columns = ['userID','gender','age','purchaseLevel']
df_item_testB = pd.read_csv('..\\data\\ECommAI_EUIR_round1_testB_20190809\\item.csv',header=None)
df_item_testB.columns = ['itemID','categoryID','shopID','brandID']
df_log_testB = pd.read_csv('..\\data\\ECommAI_EUIR_round1_testB_20190809\\user_behavior.csv',header=None)
df_log_testB.columns = ['userID','itemID','behavior','timestap']

df_log_testA['date'] = df_log_testA['timestap'].apply(lambda x : datetime.datetime(2019,7,5) + datetime.timedelta(seconds=x))
df_log_testA['day'] = df_log_testA['date'].dt.day
df_log_testA['weekday'] = df_log_testA['date'].dt.weekday + 1

df_log_testB['date'] = df_log_testB['timestap'].apply(lambda x : datetime.datetime(2019,7,5) + datetime.timedelta(seconds=x))
df_log_testB['day'] = df_log_testB['date'].dt.day
df_log_testB['weekday'] = df_log_testB['date'].dt.weekday + 1


In [6]:
df_log = pd.concat([df_log_testA,df_log_testB])
df_log.reset_index(inplace=True,drop=True)
df_log['day'].values.max()

20

In [7]:
df_log_selected = df_log[:300000].copy()  # 深拷贝
df_log_selected.loc[df_log_selected['behavior']=='pv','behavior'] = 1
df_log_selected.loc[df_log_selected['behavior']=='fav','behavior'] = 2
df_log_selected.loc[df_log_selected['behavior']=='cart','behavior'] = 3
df_log_selected.loc[df_log_selected['behavior']=='buy','behavior'] = 4

In [8]:
df_log_selected['hour'] = df_log_selected['date'].dt.hour
df_log_selected['day_hour'] = df_log_selected['day'] + df_log_selected['hour']/float(24)
df_log_selected['behavior'] = (1 - (21-df_log_selected['day_hour']+1)/(21-5+1)) * df_log_selected['behavior']
df_log_selected.head()

,userID,itemID,behavior,timestap,date,day,weekday,hour,day_hour
0,850488,28246140,0.742647,1093520,2019-07-17 15:45:20,17,3,15,17.625000
1,850488,28246140,0.745098,1095016,2019-07-17 16:10:16,17,3,16,17.666667
2,850488,28246140,0.745098,1095032,2019-07-17 16:10:32,17,3,16,17.666667
3,813599,31452441,0.54902,806429,2019-07-14 08:00:29,14,7,8,14.333333
4,581930,29175762,0.757353,1113842,2019-07-17 21:24:02,17,3,21,17.875000


In [9]:
df_user = pd.concat([df_user_testA,df_user_testB])
df_user.reset_index(inplace=True,drop=True)
df_user.head()
df_user.describe()

,userID,gender,age,purchaseLevel
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,584006.278940,0.672050,31.889050,5.647830
std,241737.547962,0.469469,9.623555,2.126474
min,45.000000,0.000000,18.000000,1.000000
25%,457785.500000,0.000000,24.000000,4.000000
50%,610173.000000,1.000000,30.000000,6.000000
75%,761068.500000,1.000000,38.000000,7.000000
max,999979.000000,1.000000,90.000000,9.000000


In [43]:
df_user.gender.value_counts()

1    67205
0    32795
Name: gender, dtype: int64

In [44]:
df_log_selected = pd.merge(df_log_selected,df_user)
len(set(df_log_selected['userID']))

44939

In [45]:
# 男性女性用户行为分开

df_log_selected_male = df_log_selected[ df_log_selected['gender']==0 ]
item_statistc_male_w = df_log_selected_male.groupby(['itemID'])[['behavior']].sum()
item_statistc_male_w.reset_index(inplace=True)
item_statistc_male_w.columns = ['itemID','itemCount_male_w']

df_log_selected_female = df_log_selected[ df_log_selected['gender']==1 ]
item_statistc_female_w = df_log_selected_female.groupby(['itemID'])[['behavior']].sum()
item_statistc_female_w.reset_index(inplace=True)
item_statistc_female_w.columns = ['itemID','itemCount_female_w']

In [46]:
df_item = pd.concat([df_item_testA,df_item_testB])
df_item.drop_duplicates(inplace=True)
df_item.reset_index(drop=True,inplace=True)
df_item.head()
df_item.shape

(1442798, 4)

In [49]:
df_item = pd.merge(df_item,item_statistc_female_w,how='left')
df_item = pd.merge(df_item,item_statistc_male_w,how='left')

df_item.loc[np.isnan(df_item['itemCount_male_w']),'itemCount_male_w' ] = 0
df_item.loc[np.isnan(df_item['itemCount_female_w']),'itemCount_female_w' ] = 0

df_item_male_sorted = df_item.sort_values(by=['itemCount_male_w'],ascending=False)
df_item_male_sorted.reset_index(drop=True,inplace=True)
item_male_w_Top500 = list( df_item_male_sorted.loc[:499,'itemID'] )



df_item_female_sorted = df_item.sort_values(by=['itemCount_female_w'],ascending=False)
df_item_female_sorted.reset_index(drop=True,inplace=True)
item_female_w_Top500 = list( df_item_female_sorted.loc[:499,'itemID'] )
df_item_female_sorted.head()

,itemID,categoryID,shopID,brandID,itemCount_female_w,itemCount_male_w
0,6078523,9170,3241560,524917,249.953431,18.580882
1,38225282,12536,912032,553129,201.928922,21.625000
2,28243760,11010,2096164,452281,192.213235,112.267157
3,41032581,5480,849187,152196,189.578431,3.710784
4,34352300,7085,2212564,40264,155.600490,28.281863


In [50]:
item_to_category_dict = dict()
for row in df_item.values:
    item_to_category_dict[row[0]] = row[1]


historicalDict = {}
for each_user in df_user_testB['userID']:
    historicalDict[each_user]= set(df_log_selected.loc[df_log_selected['userID']==each_user,'itemID'])


his_cat_Dict = {}

for each_user in df_user_testB['userID']:
    cat_list = []
    for item_t in historicalDict[each_user]:
        cat_list.append(item_to_category_dict[item_t])
    his_cat_Dict[each_user] = set(cat_list)

In [52]:
def process(each_user):

    gender_tmp = int(df_user_testB.loc[df_user_testB['userID']==each_user,'gender'])
    
    df_tmp = df_log_selected[df_log_selected['userID']==each_user]
    df_tmp.reset_index(inplace=True,drop=True)
    
    cat_his_tmp = his_cat_Dict[each_user]
    
    itemListTmp = []
    
    if len(df_tmp) > 0:
        
        item_sta = df_tmp.groupby(['itemID'])['behavior'].sum()
        item_sta = item_sta.reset_index()
        item_sta_sorted = item_sta.sort_values(by=['behavior'],ascending=False)
        item_sta_sorted.reset_index(inplace=True,drop=True)
        
        itemListTmp = itemListTmp + list(item_sta_sorted.loc[:27,'itemID'])
        

        
        if len(itemListTmp) < 50:
               
            if gender_tmp == 0:

                    for item_candidate in item_male_w_Top500:
                        if item_candidate not in itemListTmp and item_to_category_dict[item_candidate] not in cat_his_tmp:
                            itemListTmp.append(item_candidate)
                        
                        if len(itemListTmp) == 50:
                            break
            else:

                    for item_candidate in item_female_w_Top500:
                        if item_candidate not in itemListTmp and item_to_category_dict[item_candidate] not in cat_his_tmp:
                            itemListTmp.append(item_candidate)
                        
                        if len(itemListTmp) == 50:
                            break            
        
    else:
        
        if gender_tmp == 0:
            
                for item_candidate in item_male_w_Top500:
                    if item_candidate not in itemListTmp and item_to_category_dict[item_candidate] not in cat_his_tmp:
                        itemListTmp.append(item_candidate)
                    
                    if len(itemListTmp) == 50:
                        break

        else:
                for item_candidate in item_female_w_Top500:
                    if item_candidate not in itemListTmp and item_to_category_dict[item_candidate] not in cat_his_tmp:
                        itemListTmp.append(item_candidate)
                    
                    if len(itemListTmp) == 50:
                        break
    return each_user, set(itemListTmp)

In [ ]:
pool = ProcessPool(4)
res = pool.map(process, df_user_testB['userID'])
pool.close()
pool.join()
file = open('..\\prediction_result\\result.csv','w')
for element in res:
    strTmp = str(element[0]) + ',' + ','.join(map(lambda x:str(x), list(element[1]) ))
    file.write(strTmp+'\n')

file.close()